In [1]:
import hypernetx as hnx
from hypernetx import Entity
import networkx as nx
import matplotlib.pyplot as plt
import random
import time
import cProfile
import trace
import numpy as np
from collections import defaultdict
import pandas as pd
from hypernetx.utils.toys import TransmissionProblem
from hypernetx.utils.extras import HNXCount
from scipy.sparse import coo_matrix

In [2]:
# can replace this with your dataset of choice.

# transmission problem from the examples
id1 = 1
id2 = 0
edges = defaultdict(list)
for row in TransmissionProblem().df.itertuples():
    edges[row[id1]].append(row[id2])

h = hnx.Hypergraph(edges, static=False)

In [3]:
# add cell weights to each edge's property dictionary with the node ids in the edge as the keys
for edge in h.edges:
    members = h.edges[edge].elements
    cell_weights = dict()
    for i in members:
        # change this to reflect your weights of choice
        cell_weights[i] = np.random.rand()
    # add the cell weight property to the edge entity
    h.edges[edge].cellweights=cell_weights

In [4]:
# this simply outputs the cell weight of a selected node in a selected edge
weight = lambda self, node, edge: self.elements[edge].cellweights[node]

I = h.incidence_matrix(weight=weight)

In [18]:
%%time
mat = coo_matrix(I)
edges = mat.col
nodes = mat.row
weights = mat.data

edict = defaultdict(list)
celldict = defaultdict(dict)
for i in range(len(edges)):
    edict[edges[i]].append(nodes[i])
    celldict[edges[i]][nodes[i]] = weights[i]

h = hnx.Hypergraph(edict, static=False)
for e in h.edges:
    h.edges[e].cellweight = celldict[e]


CPU times: user 89.1 ms, sys: 2.8 ms, total: 91.9 ms
Wall time: 91.7 ms


In [19]:
%%time
mat = coo_matrix(I)
edges = mat.col
nodes = mat.row
weights = mat.data

edict = dict()
for i in range(len(edges)):
    try:
        edict[edges[i]].elements.append(nodes[i])
        edict[edges[i]].cellweights[nodes[i]] = weights[i]
    except:
        edict[edges[i]] = Entity(edges[i], elements=[nodes[i]], cellweights={nodes[i]: weights[i]})

h = hnx.Hypergraph(edict)

CPU times: user 192 ms, sys: 4.9 ms, total: 197 ms
Wall time: 197 ms


In [21]:
%%time
mat = coo_matrix(I)
edges = mat.col
nodes = mat.row
weights = mat.data

edict = dict()
for edge, node, weight in zip(edges, nodes, weights):
    try:
        edict[edge].add(node)
        edict[edge].cellweights[node] = weight
    except:
        edict[edge] = Entity(edge, elements=[node], cellweights={node: weight})

h = hnx.Hypergraph(edict)

CPU times: user 191 ms, sys: 5.22 ms, total: 196 ms
Wall time: 195 ms


In [22]:
%%time

verInc,edgeInc=I.nonzero() #vertex-hyperedge incidence lists

#create hyperedge dictionary
edges = defaultdict(list)
for x,y in zip(verInc,edgeInc):
    edges[y].append(x)
h = hnx.Hypergraph(edges, static=False)

# add cell weights to each edge's property dictionary with the node ids in the edge as the keys
for edge in h.edges:
    members = h.edges[edge].elements
    cell_weights = dict()
    for i in members:
        cell_weights[i] =  I[i,edge]
    # add the cell weight property to the edge entity
    h.edges[edge].cellweights=cell_weights

CPU times: user 519 ms, sys: 5.42 ms, total: 524 ms
Wall time: 525 ms
